In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.utils.data as data_utils

import numpy as np
import pandas as pd

from scipy.io import loadmat
from scipy import sparse
import os

import pickle
from tqdm import tqdm
import sys
plt.style.use('ggplot')
import gc
import warnings as w
w.filterwarnings('ignore')

### Data Preprocessing :https://www.kaggle.com/code/leejunseok97/evcf-sota-model-rc-preprocess

# now Incomplete

In [2]:
!pip install bottleneck

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.6/332.6 kB 570.8 kB/s eta 0:00:0000:0100:01


## EVCF(Enhancing VAEs for collaborative Filtering)
### Github Link : https://github.com/psywaves/EVCF

In [3]:
def load_train_data(csv_file):
    unique_sid = list()
    with open('../input/sid-data/unique_sid.txt','r') as f:
        for line in f:
            unique_sid.append(line.strip())
    tp = pd.read_csv(csv_file)
    n_users = tp['uid'].max() + 1
    n_items = len(unique_sid)

    rows, cols = tp['uid'], tp['sid']
    data = sparse.csr_matrix((np.ones_like(rows),
                                  (rows, cols)), dtype='float32',
                                 shape=(n_users, n_items)).toarray()
    return data

def load_tr_te_data(csv_file_tr, csv_file_te):
    unique_sid = list()
    with open('../input/sid-data/unique_sid.txt','r') as f:
        for line in f:
            unique_sid.append(line.strip())
    n_items = len(unique_sid)
    tp_tr = pd.read_csv(csv_file_tr)
    tp_te = pd.read_csv(csv_file_te)

    start_idx = min(tp_tr['uid'].min(), tp_te['uid'].min())
    end_idx = max(tp_tr['uid'].max(), tp_te['uid'].max())

    rows_tr, cols_tr = tp_tr['uid'] - start_idx, tp_tr['sid']
    rows_te, cols_te = tp_te['uid'] - start_idx, tp_te['sid']

    data_tr = sparse.csr_matrix((np.ones_like(rows_tr),(rows_tr, cols_tr)),
                                    dtype='float32', shape=(end_idx - start_idx + 1, n_items)).toarray()
    data_te = sparse.csr_matrix((np.ones_like(rows_te),(rows_te, cols_te)),
                                    dtype='float32', shape=(end_idx - start_idx + 1, n_items)).toarray()
    return data_tr, data_te

In [4]:
x_train = load_train_data(os.path.join("../input/netflix-preprocess", 'train (3).csv'))
np.random.shuffle(x_train)
x_val_tr, x_val_te = load_tr_te_data(os.path.join("../input/netflix-preprocess", 'validation_tr.csv'),
                                         os.path.join("../input/netflix-preprocess", 'validation_te.csv'))

x_test_tr, x_test_te = load_tr_te_data(os.path.join("../input/netflix-preprocess", 'test_tr.csv'),
                                           os.path.join("../input/netflix-preprocess", 'test_te.csv'))

# idle y's
y_train = np.zeros((x_train.shape[0], 1))


# pytorch data loader
train = data_utils.TensorDataset(torch.from_numpy(x_train), torch.from_numpy(y_train))
train_loader = data_utils.DataLoader(train, batch_size=512, shuffle=True)

validation = data_utils.TensorDataset(torch.from_numpy(x_val_tr), torch.from_numpy(x_val_te))
val_loader = data_utils.DataLoader(validation, batch_size=512, shuffle=False)

test = data_utils.TensorDataset(torch.from_numpy(x_test_tr).float(), torch.from_numpy(x_test_te))
test_loader = data_utils.DataLoader(test, batch_size=512, shuffle=True)

In [5]:
data,label = next(iter(train_loader))
print(data.shape,label.shape)

torch.Size([512, 4497]) torch.Size([512, 1])


In [6]:
sys.path.insert(0,os.path.abspath('../input/-utils'))

In [7]:
sys.path

['/kaggle/input/-utils',
 '/kaggle/working',
 '/kaggle/lib/kagglegym',
 '/kaggle/lib',
 '/opt/conda/lib/python37.zip',
 '/opt/conda/lib/python3.7',
 '/opt/conda/lib/python3.7/lib-dynload',
 '',
 '/root/.local/lib/python3.7/site-packages',
 '/opt/conda/lib/python3.7/site-packages',
 '/src/bq-helper',
 '/opt/conda/lib/python3.7/site-packages/IPython/extensions',
 '/root/.ipython']

In [8]:
import torch
import torch.utils.data
import torch.nn as nn
from torch.nn import Linear
from torch.autograd import Variable
from torch.nn.functional import normalize

from Optimizer import AdamNormGrad
import bottleneck as bn
from nn import he_init, GatedDense, NonLinear
from distribution import log_Bernoulli, log_Normal_diag, log_Normal_standard, log_Logistic_256, log_Softmax

In [9]:
class Model(nn.Module):
    def __init__(self,input_size=[1,28,28],number_components=1000,
                 pseudoinputs_mean=0.05,pseudoinputs_std=0.01,use_training_data_init=False):
        super(Model, self).__init__()
        self.input_size = input_size
        self.number_components = number_components
        self.pseudoinputs_mean = pseudoinputs_mean
        self.pseudoinputs_std = pseudoinputs_std
        self.use_training_data_init = use_training_data_init
    # AUXILIARY METHODS
    def add_pseudoinputs(self):

        nonlinearity = nn.Hardtanh(min_val=0.0, max_val=1.0)

        self.means = NonLinear(self.number_components, np.prod(self.input_size), bias=False, activation=nonlinearity)

        # init pseudo-inputs
        if self.use_training_data_init:
            self.means.linear.weight.data = self.pseudoinputs_mean
        else:
            normal_init(self.means.linear, self.pseudoinputs_mean, self.pseudoinputs_std)

        # create an idle input for calling pseudo-inputs
        self.idle_input = Variable(torch.eye(self.number_components, self.number_components), requires_grad=False)
        
    def reparameterize(self, mu, logvar):
        if self.training:
            std = logvar.mul(0.5).exp_()
            
            eps = torch.cuda.FloatTensor(std.size()).normal_()
            
            eps = Variable(eps)
            return eps.mul(std).add_(mu)
        else:
            return mu

    def calculate_loss(self):
        return 0.

    def calculate_likelihood(self):
        return 0.

    def calculate_lower_bound(self):
        return 0.

    # THE MODEL: FORWARD PASS
    def forward(self, x):
        return 0.


In [10]:
class VAE(Model):
    def __init__(self,input_size,hidden_size,gated,num_layers,z1_size):
        super(VAE,self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.gated = gated
        self.num_layers = num_layers
        self.z1_size = z1_size
        modules = [
            nn.Dropout(0.5),
            NonLinear(np.prod(self.input_size), self.hidden_size, 
                      gated=self.gated,activation=nn.Tanh())
        ]
        for _ in range(0, self.num_layers - 1):
            modules.append(
                NonLinear(self.hidden_size,self.hidden_size,
                        gated=self.gated,activation=nn.Tanh()))
        self.q_z_layers = nn.Sequential(*modules)
        
        self.q_z_mean = Linear(self.hidden_size,self.z1_size)
        self.q_z_logvar = NonLinear(self.hidden_size,self.z1_size,
                                    activation=nn.Hardtanh(min_val=-12.,max_val=4.))
        
        
        modules = [NonLinear(self.z1_size,self.hidden_size,
                             gated=self.gated,activation=nn.Tanh())]
        for _ in range(0,self.num_layers - 1):
            modules.append(
                NonLinear(self.hidden_size,self.hidden_size,
                          gated=self.gated,activation=nn.Tanh()))
        self.p_x_layers = nn.Sequential(*modules)
        
        self.p_x_mean = NonLinear(self.hidden_size,np.prod(self.input_size),
                                 activation=None)
        
        for m in self.modules():
            if isinstance(m,nn.Linear):
                he_init(m)
                
    def calculate_loss(self,x,beta=1.,average=False):
        
        x_mean, x_logvar, z_q, z_q_mean, z_q_logvar = self.forward(x)
        
        RE = log_Softmax(x,x_mean,dim=1)
        
        
        log_p_z = self.log_p_z(z_q)
        log_q_z = log_Normal_diag(z_q, z_q_mean, z_q_logvar,dim=1)
        KL = -(log_p_z - log_q_z)
        
        loss = -RE + beta * KL
        
        if average:
            loss = torch.mean(loss)
            RE = torch.mean(RE)
            KL = torch.mean(KL)
            
        return loss, RE, KL
    
    def reconstruct_x(self,x):
        x_mean, _, _, _, _ = self.forward(x)
        return x_mean
    
    def q_z(self,x):
        x = self.q_z_layers(x)
        
        z_q_mean = self.q_z_mean(x)
        z_q_logvar = self.q_z_logvar(x)
        
        return z_q_mean, z_q_logvar
    
    def p_x(self,z):
        z = self.p_x_layers(z)
        
        x_mean = self.p_x_mean(z)
        x_logvar = 0
        
        return x_mean, x_logvar
    
    def log_p_z(self,z):
        log_prior = log_Normal_standard(z,dim=1)
        
        return log_prior
    
    def forward(self,x):
        
        x = normalize(x,dim=1)
        
        z_q_mean, z_q_logvar = self.q_z(x)
        
        z_q = self.reparameterize(z_q_mean,z_q_logvar)
        
        x_mean, x_logvar = self.p_x(z_q)
        
        return x_mean, x_logvar, z_q, z_q_mean, z_q_logvar
                           

In [11]:
batch_size = 512
epoch = 400
lr = 5e-3
warmup = 100
max_beta = 1

num_layers = 1
z1_size = 200
z2_size = 200
hidden_size = 600
input_size = 4497
device = ('cuda' if torch.cuda.is_available() else 'cpu')
mode = 'train'

In [12]:
model = VAE(input_size=input_size,hidden_size=hidden_size,gated=True,
            num_layers=num_layers,z1_size=z1_size)
optimizer = AdamNormGrad(model.parameters(),lr=lr)
model.to(device)
model

VAE(
  (q_z_layers): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): NonLinear(
      (activation): Tanh()
      (linear): Linear(in_features=4497, out_features=600, bias=True)
      (sigmoid): Sigmoid()
      (g): Linear(in_features=4497, out_features=600, bias=True)
    )
  )
  (q_z_mean): Linear(in_features=600, out_features=200, bias=True)
  (q_z_logvar): NonLinear(
    (activation): Hardtanh(min_val=-12.0, max_val=4.0)
    (linear): Linear(in_features=600, out_features=200, bias=True)
  )
  (p_x_layers): Sequential(
    (0): NonLinear(
      (activation): Tanh()
      (linear): Linear(in_features=200, out_features=600, bias=True)
      (sigmoid): Sigmoid()
      (g): Linear(in_features=200, out_features=600, bias=True)
    )
  )
  (p_x_mean): NonLinear(
    (linear): Linear(in_features=600, out_features=4497, bias=True)
  )
)

In [13]:
def NDCG_binary_at_k_batch(X_pred, heldout_batch,k=100):
    batch_users = X_pred.shape[0]
    idx_topk_part = bn.argpartition(-X_pred,k,axis=1)
    topk_part = X_pred[np.arange(batch_users)[:,np.newaxis],idx_topk_part[:,:k]]
    idx_part = np.argsort(-topk_part, axis=1)
    
    idx_topk = idx_topk_part[np.arange(batch_users)[:, np.newaxis],idx_part]
    tp = 1. / np.log2(np.arange(2,k+2))
    tp = torch.tensor(tp,dtype=torch.float)
    
    DCG = (heldout_batch[np.arange(batch_users)[:,np.newaxis],idx_topk].cpu() * tp).sum(dim=1)
    
    IDCG = torch.tensor([(tp[:min(n,k)]).sum() for n in (heldout_batch != 0).sum(dim=1)])
    
    return DCG / IDCG
    

In [14]:
def Recall_at_k_batch(X_pred,heldout_batch,k=100):
    batch_users = X_pred.shape[0]
    
    idx = bn.argpartition(-X_pred,k,axis=1)
    X_pred_binary = np.zeros_like(X_pred,dtype=bool)
    X_pred_binary[np.arange(batch_users)[:,np.newaxis], idx[:,:k]] = True
    
    X_true_binary = torch.tensor((heldout_batch > 0),dtype=torch.float)
    tmp = torch.tensor(np.logical_and(X_true_binary,X_pred_binary),dtype=torch.float).sum(dim=1)
    recall = tmp / np.minimum(k,X_true_binary.sum(dim=1))
    return recall

In [ ]:
for e in range(epoch):
    train_loss = 0
    train_re = 0
    train_kl = 0
    
    valid_loss = 0
    valid_re = 0
    valid_kl = 0
    
    model.train()
    
    if warmup == 0:
        beta = max_beta
    else:
        beta = max_beta * epoch / warmup
        if beta > max_beta:
            beta = max_beta
    print(f'beta: {beta}')
    
    for batch_idx, (data,target) in tqdm(enumerate(train_loader)):
        data, target = data.to(device), target.to(device)
        data, target = Variable(data), Variable(target)
        
        optimizer.zero_grad()
        
        loss, RE, KL = model.calculate_loss(data,beta,average=True)
        
        loss.backward()
        optimizer.step()
        
        train_loss += loss.data.item()
        train_re += -RE.data.item()
        train_kl += KL.data.item()
    train_loss /= len(train_loader)
    train_re /= len(train_loader)
    train_kl /= len(train_loader)
    
    
    ndcg_dist = torch.tensor([],dtype=torch.float)
#     if mode == 'test':
#         ndcg_20 = torch.tensor([], dtype=torch.float)
#         ndcg_10 = torch.tensor([], dtype=torch.float)
#         recall_50 = torch.tensor([], dtype=torch.float)
#         recall_20 = torch.tensor([], dtype=torch.float)
#         recall_10 = torch.tensor([], dtype=torch.float)
#         recall_5 = torch.tensor([], dtype=torch.float)
#         recall_1 = torch.tensor([], dtype=torch.float)
    model.eval()
        
    for batch_idx, (valid_tr,valid_te) in tqdm(enumerate(val_loader)):
        valid_tr, valid_te = valid_tr.to(device), valid_te.to(device)
        
        valid_tr, valid_te = Variable(valid_tr), Variable(valid_te)
        
        with torch.no_grad():
            loss , RE, KL = model.calculate_loss(valid_tr,average=True)
            
            valid_loss += loss.data.item()
            valid_re = -RE.data.item()
            valid_kl = KL.data.item()
            
            
            pred_val = model.reconstruct_x(valid_tr).detach().cpu().numpy()
            
            valid_tr = valid_tr.detach().cpu().numpy()
            
            pred_val[valid_tr.nonzero()] = -np.Inf
            
            ndcg_dist = torch.cat([ndcg_dist, NDCG_binary_at_k_batch(pred_val, valid_te,k=100)])
            
            
#             if mode == 'test':
#                 ndcg_20 = torch.cat([ndcg_20, NDCG_binary_at_k_batch(pred_val, test, k=20)])
#                 ndcg_10 = torch.cat([ndcg_10, NDCG_binary_at_k_batch(pred_val, test, k=10)])
#                 recall_50 = torch.cat([recall_50, Recall_at_k_batch(pred_val, test, k=50)])
#                 recall_20 = torch.cat([recall_20, Recall_at_k_batch(pred_val, test, k=20)])
#                 recall_10 = torch.cat([recall_10, Recall_at_k_batch(pred_val, test, k=10)])
#                 recall_5 = torch.cat([recall_5, Recall_at_k_batch(pred_val, test, k=5)])
#                 recall_1 = torch.cat([recall_1, Recall_at_k_batch(pred_val, test, k=1)])
        valid_loss /= len(val_loader)
        valid_re /= len(val_loader)
        valid_kl /= len(val_loader)
        
        evaluate_ndcg = ndcg_dist.mean().data.item()
        
        
#         if mode == 'test':
#             eval_ndcg100 = "{:.5f}({:.4f})".format(evaluate_ndcg, ndcg_dist.std().data.item()/np.sqrt(len(ndcg_dist)))
#             eval_ndcg20 = "{:.5f}({:.4f})".format(ndcg_20.mean().data.item(),ndcg_20.std().data.item()/np.sqrt(len(ndcg_20)))
#             eval_ndcg10 = "{:.5f}({:.4f})".format(ndcg_10.mean().data.item(),ndcg_10.std().data.item()/np.sqrt(len(ndcg_10)))
#             eval_recall50 = "{:.5f}({:.4f})".format(recall_50.mean().data.item(),recall_50.std().data.item()/np.sqrt(len(recall_50)))
#             eval_recall20 = "{:.5f}({:.4f})".format(recall_20.mean().data.item(),recall_20.std().data.item()/np.sqrt(len(recall_20)))
#             eval_recall10 = "{:.5f}({:.4f})".format(recall_10.mean().data.item(),recall_10.std().data.item()/np.sqrt(len(recall_10)))
#             eval_recall5 = "{:.5f}({:.4f})".format(recall_5.mean().data.item(),recall_5.std().data.item()/np.sqrt(len(recall_5)))
#             eval_recall1 = "{:.5f}({:.4f})".format(recall_1.mean().data.item(),recall_1.std().data.item()/np.sqrt(len(recall_1)))
    print('Epoch:{} Train_Loss:{:.3f}\tTrain RE:{:.3f}\tTrain KL:{:.3f}'.format(e+1,train_loss,train_re,train_kl))
    print('Epoch:{} Valid_Loss:{:.3f}\tValid RE:{:.3f}\tValid KL:{:.3f}'.format(e+1,valid_loss,valid_re,valid_kl))
            

beta: 1


547it [00:09, 57.61it/s]
79it [00:08,  9.27it/s]


Epoch:1 Train_Loss:238.007	Train RE:232.779	Train KL:5.228
Epoch:1 Valid_Loss:2.900	Valid RE:2.758	Valid KL:0.111
beta: 1


547it [00:08, 67.23it/s]
79it [00:07,  9.95it/s]


Epoch:2 Train_Loss:231.122	Train RE:225.334	Train KL:5.788
Epoch:2 Valid_Loss:2.874	Valid RE:2.730	Valid KL:0.114
beta: 1


...

547it [00:08, 64.27it/s]
79it [00:07,  9.98it/s]


Epoch:19 Train_Loss:228.606	Train RE:222.250	Train KL:6.356
Epoch:19 Valid_Loss:2.866	Valid RE:2.710	Valid KL:0.126
beta: 1


547it [00:08, 67.41it/s]
79it [00:07, 10.33it/s]


Epoch:20 Train_Loss:228.728	Train RE:222.378	Train KL:6.350
Epoch:20 Valid_Loss:2.863	Valid RE:2.707	Valid KL:0.125
beta: 1


547it [00:08, 64.64it/s]
61it [00:05, 10.68it/s]